
# 1 - Imports and defining functions

In [52]:
import sys
import os

meshplot_folder = '/home/s94zalek/shape_matching/meshplot_fork'
sys.path.append(meshplot_folder)
os.chdir(meshplot_folder)
import meshplot as mp

base_folder = '/home/s94zalek/shape_matching/pyFM_fork'

sys.path.append(base_folder)
os.chdir(base_folder)

In [53]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

# 2- Loading meshes to match

In [54]:
mesh1 = TriMesh('examples/data/cat-00.off', center=True, area_normalize=True).process(k=150, intrinsic=True)
mesh2 = TriMesh('examples/data/lion-00.off', center=True, area_normalize=True).process(k=150, intrinsic=True)

# 3 - Computing Initial Correspondences

ZoomOut works using initial correspondences.
We show here two versions:
 - One using Nearest Neighbor initialization (hoping that meshes are aligned)
 - One using WKS initialization


**NOTE**: In this package, *functional map* usually go from Shape $1$ to Shape $2$, and *pointwise maps* from shape $2$ to shape $1$.


Let's compute the initial maps from `mesh2` to `mesh1`

In [55]:
from pyFM.signatures.WKS_functions import mesh_WKS
from pyFM.spectral.nn_utils import knn_query

wks_descr1 = mesh_WKS(mesh1, num_E=100, k=100)
wks_descr2 = mesh_WKS(mesh2, num_E=100, k=100)

# WKS initialisation
p2p_21_wks = knn_query(wks_descr1, wks_descr2, k=1) # (n2,) initialisation with 1-NN

# NN Initialisation
p2p_21_nn = knn_query(mesh1.vertices, mesh2.vertices, k=1) # (n2,) initialisation with 1-NN

In [ ]:
cmap1 = visu(mesh1.vertlist)
cmap2_wks = cmap1[p2p_21_wks]
cmap2_nn = cmap1[p2p_21_nn]

double_plot(mesh1,mesh2,cmap1,cmap2_wks)

# 4 - Applying ZoomOut

We can remesh this mapn using the [ZoomOut algorithm](https://www.lix.polytechnique.fr/~maks/papers/papers_235s4-file1.pdf).

This iterative algorithm refines functional maps but can easily be adapted to refine pointwise correspondences

In [59]:
from pyFM.refine.zoomout import mesh_zoomout_refine_p2p

In [60]:
FM_12_wks_zo, p2p_21_wks_zo = mesh_zoomout_refine_p2p(p2p_21=p2p_21_wks, mesh1=mesh1, mesh2=mesh2, k_init=20, nit=16, step=5, return_p2p=True, n_jobs=10, verbose=True)
FM_12_nn_zo, p2p_21_nn_zo = mesh_zoomout_refine_p2p(p2p_21=p2p_21_nn, mesh1=mesh1, mesh2=mesh2, k_init=20, nit=16, step=5, return_p2p=True, n_jobs=10, verbose=True)

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

In [ ]:
cmap1 = visu(mesh1.vertlist)
cmap2_wks = cmap1[p2p_21_wks_zo]
cmap2_nn = cmap1[p2p_21_nn_zo]

double_plot(mesh1,mesh2,cmap1,cmap2_nn)